In [11]:
import zipfile
z = zipfile.ZipFile("catsdogs.zip")
z.extractall()

# Cats and Dogs: Data Preprocessing

In [12]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
from tensorflow.keras.preprocessing.image import load_img
warnings.filterwarnings("ignore")

In [24]:
X = []
Y= []
for i in os.listdir("PetImages"):
  for j in os.listdir("PetImages/"+i):
    if i ==  "Cat":
      Y.append(0)
    else:
      Y.append(1)
    X.append(os.path.join("PetImages",i,j))

print(X[0],Y[0])

PetImages/Dog/8323.jpg 1


In [27]:
dfX = pd.DataFrame(X)
dfY = pd.DataFrame(Y)
df = pd.concat([dfX,dfY],axis=1)
df.columns = ["images","label"]

In [28]:
df.head()

,images,label
0,PetImages/Dog/8323.jpg,1
1,PetImages/Dog/6495.jpg,1
2,PetImages/Dog/5974.jpg,1
3,PetImages/Dog/5833.jpg,1
4,PetImages/Dog/2310.jpg,1


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25002 entries, 0 to 25001
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   images  25002 non-null  object
 1   label   25002 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 390.8+ KB


In [32]:
#Find if there is any file which is not opened
for i in df["images"]:
  if  i[-4:] != ".jpg":
    print(i)

PetImages/Dog/Thumbs.db
PetImages/Cat/Thumbs.db


In [34]:
#create a list for trash images
import PIL
trash = []
for i in df["images"]:
    try:
        img = PIL.Image.open(i)
    except:
        trash.append(i)
trash

['PetImages/Dog/11702.jpg',
 'PetImages/Dog/Thumbs.db',
 'PetImages/Cat/666.jpg',
 'PetImages/Cat/Thumbs.db']

In [35]:
#delete the files which is not opened
for i in trash:
    df = df[df["images"] != i]

In [41]:
#resize all files
from PIL import Image
import cv2
import os
for index,link in enumerate(df["images"][:12499]):
    images = Image.open(link).convert('RGB')
    images = images.resize((500,500),Image.ANTIALIAS)

for index,link in enumerate(df["images"][12499:]):
    images = Image.open(link).convert('RGB')
    images = images.resize((500,500),Image.ANTIALIAS)

In [45]:
df_X = pd.DataFrame(X)
df_Y= pd.DataFrame(Y)
df = pd.concat([df_X,df_Y],axis=1)
df.columns = ["images","label"]
df.head()

,images,label
0,PetImages/Dog/8323.jpg,1
1,PetImages/Dog/6495.jpg,1
2,PetImages/Dog/5974.jpg,1
3,PetImages/Dog/5833.jpg,1
4,PetImages/Dog/2310.jpg,1


In [53]:
print("Lenght of samples:" ,len(X))
print("Lenght of labels:", len(Y))

Lenght of samples: 25002
Lenght of labels: 25002


# 2) Modelling

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [74]:
model.compile(
    optimizer = 'adam', 
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['acc']
)

In [88]:
from keras.preprocessing.image import ImageDataGenerator
#ImageDataGenerator dökümanından hazır olarak alındı.
train_generator = ImageDataGenerator(
    rescale = 1./255,  # normalization of images
    rotation_range = 40, # augmention of images to avoid overfitting
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

val_generator = ImageDataGenerator(rescale = 1./255)

train_iterator = train_generator.flow_from_dataframe(
    train, 
    x_col='images', 
    y_col='label', 
    target_size=(128,128), 
    batch_size=512, 
    class_mode='binary'
)

val_iterator = val_generator.flow_from_dataframe(
    test, 
    x_col='images', 
    y_col='label', 
    target_size=(128,128), 
    batch_size=512, 
    class_mode='binary'
)

Found 16749 validated image filenames belonging to 2 classes.
Found 8251 validated image filenames belonging to 2 classes.


In [91]:
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
model = Sequential([
                    Conv2D(16, (3,3), activation='relu', input_shape=(128,128,3)),
                    MaxPool2D((2,2)),
                    Conv2D(32, (3,3), activation='relu'),
                    MaxPool2D((2,2)),
                    Conv2D(64, (3,3), activation='relu'),
                    MaxPool2D((2,2)),
                    Flatten(),
                    Dense(512, activation='relu'),
                    Dense(1, activation='sigmoid')
])

In [93]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 64)      

In [96]:
history = model.fit(train_iterator, epochs=10, validation_data=val_iterator)

Epoch 1/10
 5/33 [===>..........................] - ETA: 3:50 - loss: 0.6950 - accuracy: 0.5105

UnknownError: ignored

In [95]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Accuracy Graph')
plt.legend()
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Loss Graph')
plt.legend()
plt.show()

NameError: ignored